# Exercice 1#


# 1) 
Dans cette première partie notre objectif est de déterminer sur un ensemble de données
un estimateur linéaire de la concentration en ozone à partir des variables suivantes :
* le rayonnement solaire
* la force du vent 
* la température
* le jour de la mesure
* le mois de la mesure

soit le modèle linéaire avec condition gaussienne :

$$Y = X\theta + \epsilon~~ où~~ X~~ est~~ une~~ matrice~~ (n,p)~~ de~~ rang~~ p,\\
Y\in\mathbb{R}^n\;\;et\;\;
\theta~~ est~~ un~~ vecteur~~ inconnu~~ de~~ \mathbb{R}^p\\
~~ \epsilon~~ est~~ un~~ vecteur~~ de~~ n~~ réalisations~~ indépendantes~~ d’une~~ v.a.\\
gaussienne~~ de~~ moyenne~~ 0~~ et~~ de~~ variance~~ \sigma^2*\mathsf{I}d\\
tel~~ que~~ \epsilon\sim\mathcal{N}(0_n,\sigma^2*\mathsf{I}d)\\
on~~ peut~~ aussi~~ noter~~ y_i~~ =~~ {\theta}_0^{*} + \sum_{j=1}^{p} {\theta}_j^{*}x_{ij} + {\epsilon}_i\\
avec~~ {\epsilon}_i = {\cal{N}}(0,{\sigma}^2)~~ \forall i = 1,...,n$$



# 2)


In [1]:
# -*- coding: utf-8 -*-
#
#Importation des librairies nécessaires
#
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import statsmodels.datasets as sm
from IPython.display import display, Math, Latex
import pylab
import numpy.random as npr

dataset = sm.get_rdataset('airquality').data
dataset.head()


,Ozone,Solar.R,Wind,Temp,Month,Day
0,41,190,7.4,67,5,1
1,36,118,8.0,72,5,2
2,12,149,12.6,74,5,3
3,18,313,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


# 3)

In [2]:
dataset = dataset.dropna()
dataset.head()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41,190,7.4,67,5,1
1,36,118,8.0,72,5,2
2,12,149,12.6,74,5,3
3,18,313,11.5,62,5,4
6,23,299,8.6,65,5,7


# 4)

In [32]:
from sklearn import linear_model
y_train = dataset['Ozone']
x_train = dataset[['Solar.R','Wind','Temp','Month','Day']]
x_scaled = (x_train - np.mean(x_train))/np.std(x_train)
skl_linmod_ozone_scaled = linear_model.LinearRegression()
skl_linmod_ozone_scaled.fit(x_scaled,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

# 5)

In [33]:
theta_hat = np.insert(skl_linmod_ozone_scaled.coef_,0,skl_linmod_ozone_scaled.intercept_,axis=None)
display(Math(r'$\widehat\theta = \begin{bmatrix}'+r'{0}'.format(theta_hat[0])+r'\\{0}'.format(theta_hat[1])+r'\\{0}'.format(theta_hat[2])+r'\\{0}'.format(theta_hat[3])+r'\\{0}'.format(theta_hat[4])+r'\\{0}'.format(theta_hat[5])+r'\end{bmatrix}$'))


<IPython.core.display.Math object>

l'estimateur sans biais de la variance $$\hat{\sigma^2} = \frac{\Vert~ y-\hat{y}\Vert^2}{n-rang(X)}\
où~~ y~~ -~~ \hat{y}~~ est~~ le~~ vecteur~~ résidus~~ avec~~ \hat{y} = X\hat{\theta}$$ 

In [34]:
y_hat = skl_linmod_ozone_scaled.predict(x_scaled) 
residus = y_train - y_hat  
norm_2 = residus.T.dot(residus) #calcul de la norme carré des résidus
n = len(y_train)
p = np.linalg.matrix_rank(x_scaled)
var_hat = norm_2/(n - p)
#l'estimateur sans biais de la variance #
display(Math(r'$\hat{\sigma^2}= '+ str(var_hat)))

<IPython.core.display.Math object>

# 6)

In [35]:
#d'apres la prop 1 on peut utiliser la Statistiques de Student avec theta_hat et var_hat
skl_linmod_ozone_original = linear_model.LinearRegression()
skl_linmod_ozone_original.fit(x_train,y_train)
theta_o = skl_linmod_ozone_original.coef_
theta_o_zero = skl_linmod_ozone_original.intercept_
#print theta_o_zero
#print theta_o
theta_hat_etoile = np.insert(theta_o,0,theta_o_zero,axis=None)
#print fu
x_new = np.asarray(x_train)
x_new = np.insert(x_new,0,np.ones(n),axis=1)


from scipy.stats import t

alpha = 1 - 0.99
t_q = t.ppf(1 - alpha/2,n-p-1)


def left(i):
    """fonction qui calcule la borne inférieure de l'intervalle de confiance pour theta_hat(i)"""
    return theta_hat_etoile[i] - t_q*np.math.sqrt(var_hat*np.linalg.solve(x_new.T.dot(x_new),np.identity(p+1))[i][i])

def right(i):
    """fonction qui calcule la borne suppérieure de l'intervalle de confiance pour theta_hat(i)"""
    return theta_hat_etoile[i] + t_q*np.math.sqrt(var_hat*np.linalg.solve(x_new.T.dot(x_new),np.identity(p+1))[i][i])

def IC(i):
    """intervalle de confiance pour theta_hat(i)"""
    return "[ "+str(left(i))+";"+str(right(i)) + "]"
def IC_size(i):
    """taille de l'intervalle de confiance pour theta_hat(i)"""
    return right(i) - left(i)




#on affiche les intervalles de confiance pour les différents theta_hat(i) avec i=0...4
display(Math(r'$IC^*_0 ={}$'.format(IC(0))))
display(Math(r'$IC^*_1 ={}$'.format(IC(1))))
display(Math(r'$IC^*_2 ={}$'.format(IC(2))))
display(Math(r'$IC^*_3 ={}$'.format(IC(3))))
display(Math(r'$IC^*_4 ={}$'.format(IC(4))))
display(Math(r'$IC^*_5 ={}$'.format(IC(5))))



print IC_size(0),IC_size(1),IC_size(2),IC_size(3),IC_size(4)
print IC_size(5)
print " "
print theta_hat_etoile


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

122.628436468 0.122294820479 3.36570789442 1.43027089251 7.9034571043
1.19936908113
 
[ -6.41163211e+01   5.02743188e-02  -3.31844386e+00   1.89578642e+00
  -3.03995664e+00   2.73877521e-01]


# 7)

$$On~~ remarque~~ que~~ les~~ variables~~ Day~~ et~~ Month\\ 
ne~~ sont~~ pas~~ pertinentes~~ étant~~ donné~~ que~~ \hat{\theta_4}~~ et~~ \hat{\theta_5}~~ sont~~ trop~~ proches~~ des~~ bornes~~ limites~~
$$

# 8)

In [36]:
#Création de la nouvelle observation
listenom = ['Solar.R','Wind','Temp','Month','Day']
newline = pd.DataFrame([[197, 10, 70, 1,3]], columns = listenom)

#prédiction de la valeur d'Ozone pour la nouvelle observation
y_newline_pred = skl_linmod_ozone_original.predict(newline)
print "La prédiction pour cette nouvelle observation donne : " 
display(Math(r'$\hat{y}_{new}~~ =~~ '+str(y_newline_pred[0])))

La prédiction pour cette nouvelle observation donne : 


<IPython.core.display.Math object>

# Exercice 2 #

# 1)

In [186]:
from sklearn.utils import resample
